**Importing Libraries**


In [21]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

from plotly import tools
import plotly.offline as pyo
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

**Data Overview**
Let's have a look at the data first. Reading sheet1 from the input Excel

In [2]:
os.listdir('/kaggle/input')
df = pd.read_excel('/kaggle/input/Customer complaints 2014 through 2019.xlsx', sheet_name='Sheet1')
len(df.columns.values)

60

Setting the Display options 

In [3]:

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None 

 Removing columns which have column name **'Ignore this column/no data'** as they don't have data in most of the rows. Changed the Column name to second row as the first row have meta data related to the column names. Created a new column **Total Cost**(quantity multiplied price)

In [4]:
df.drop(df.filter(regex=("Ignore this column.*")).columns, axis = 1, inplace = True)
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df.loc[:,'Total Cost'] = df['Ref. quantity']*df['Reference number']

The input after filtering has a shape (898, 59) and below are the sample records

In [5]:
df.head(3)

,Notification,Required Start,Required End,Notification Status,Priority text,Description,Material,Customer,Complaint quantity,Batch,Catalog profile,Changed On,City,Client,Code group,Code group text,Coding Code,Coding code text,Completion by date,Completion time,Country Key,Created at,Created On,Customer Material Number,Defective (external),Defective (internal),District,Malfunction start,Material Avail. Date,Notification date,Notification origin,Notification Time,Notification type,Plant for material,Postal Code,Priority,Priority type,Processing time,Processing time unit,Production Machine(s),Purchase Order Date,Purchase Order Number,Ref. quantity,Reference date,Reference number,Reference time,Region,Required End Time,Required Start Time,Return delivery qty,Sales Order,Sales order item,Sales Organization,Scrap cost,Serial Number,Start of Malfunctn (Time),Time of change,Unit of measure,Vendor Batch,Total Cost
1,200053540,2014-01-07 00:00:00,2014-01-28 00:00:00,NOCO NOTI ATCO NOPT,Major,Missing actuator within AQL - Accepted,10308487,1201843,0,NaN,QM0000001,2014-02-03 00:00:00,ORMOND BEACH,100,Z004,PRODUCT ISSUE,Z400,ASSEMBLY RELATED,2014-02-03 00:00:00,16:06:43,US,21:16:36,2014-01-07 00:00:00,NaN,0,0,NaN,2014-01-07 00:00:00,NaN,2014-01-07 00:00:00,Q1,21:28:42,ZC,2004,32174,2,ZQ,642.630,H,NaN,2013-01-30 00:00:00,2550012769,0,2014-02-03 00:00:00,271.150,16:06:43,FL,21:28:42,21:28:42,0,1112925,0,2000,0,NaN,21:29:47,21:06:45,PC,NaN,0.000
2,200053571,2014-01-08 00:00:00,2014-01-29 00:00:00,NOCO NOTI ATCO NOPT,Major,Incorrect box count-shortage,10311413,1009654,1170,0003285770,QM0000001,2014-05-30 00:00:00,LACHINE,100,NaN,NaN,NaN,NaN,2014-05-30 00:00:00,15:40:45,CA,14:45:07,2014-01-08 00:00:00,NaN,0,0,NaN,2014-01-08 00:00:00,2013-06-07 00:00:00,2014-01-08 00:00:00,Q1,14:51:57,ZC,2004,H8T 3M5,2,ZQ,3408.810,H,AS00M162,2013-04-23 00:00:00,02-0191752,1170,2014-05-30 00:00:00,187.480,13:40:45,QC,14:51:57,14:51:57,0,1127709,10,2000,0,NaN,14:57:16,13:40:49,PC,NaN,219351.600
3,200053628,2014-01-09 00:00:00,2014-01-30 00:00:00,NOCO NOTI ATCO NOPT,Major,VP4E pumps with missing dip tubes,10065555,1010652,5031,NaN,QM0000001,2015-01-13 00:00:00,HOLMDEL,100,Z004,PRODUCT ISSUE,Z400,ASSEMBLY RELATED,2015-01-13 00:00:00,15:54:57,US,20:53:04,2014-01-09 00:00:00,NaN,0,0,NaN,2014-01-09 00:00:00,2013-07-08 00:00:00,2014-01-09 00:00:00,Q1,20:59:29,ZC,2004,07733,2,ZQ,8850.920,H,NaN,2013-05-03 00:00:00,H051276,5031,2015-01-13 00:00:00,1841.350,14:54:57,NJ,20:59:29,20:59:29,0,1129356,10,2000,0,NaN,21:19:27,14:55:01,PC,NaN,9263831.850


Below we will group various paramaters(defect processing time, quantity etc) wrt to product and customer details.
First plot we look at sum of the defect processing time taken for each Issue

In [22]:
perCodeProcessingTime = df.groupby(['Coding code text'])[['Processing time']].sum().sort_values(by='Processing time',ascending=False )
perCodeProcessingTime['CumulativePercent'] = (perCodeProcessingTime['Processing time'].cumsum()/perCodeProcessingTime['Processing time'].sum())*100
perCodeProcessingTime

,Processing time,CumulativePercent
Coding code text,,
ASSEMBLY RELATED,827403.860,30.727
MOLDING RELATED,358938.890,44.056
PROCEDURE NOT FOLLOWED,324220.220,56.096
UNJUSTIFIED ISSUE,322098.400,68.058
ORDER ENTRY ERROR,220021.740,76.229
INTERNAL SHIPPING ERROR,198305.300,83.593
PACKING LINE ERROR,141543.470,88.849
VENDOR ERROR,85765.810,92.034
INTERCO RELATED,62949.840,94.372


In [20]:
trace0 = go.Scatter(x = perCodeProcessingTime.index,y = perCodeProcessingTime['Processing time'],
                   name = "Defect Processing Time")
data = [trace0]
layout = go.Layout(title = "Processing time(SUM) per Different Issue")
fig  = go.Figure(data=data,layout = layout)
iplot(fig,filename = 'file1')

Now we will see how the ProcessingTime(time taken to resolve the Issue) distribution is for each individual Issue Type.

In [8]:
AssemblyRelated = df[df['Coding code text'] == 'ASSEMBLY RELATED'][['Processing time']]                    
MoldingRelated = df[df['Coding code text'] == 'MOLDING RELATED'][['Processing time']]                         
ProcedureNotFollowed = df[df['Coding code text'] == 'PROCEDURE NOT FOLLOWED '][['Processing time']]                  
UnjustifiedIssue = df[df['Coding code text'] == 'UNJUSTIFIED ISSUE'][['Processing time']]                        
OrderEntryError =  df[df['Coding code text'] == 'ORDER ENTRY ERROR'][['Processing time']]                        
InternalShippingError =  df[df['Coding code text'] == 'INTERNAL SHIPPING ERROR'][['Processing time']]                
PackingLineError =  df[df['Coding code text'] == 'PACKING LINE ERROR'][['Processing time']]                     
VendorError =  df[df['Coding code text'] == 'VENDOR ERROR'][['Processing time']]                         
IntercoRelated =  df[df['Coding code text'] == 'INTERCO RELATED'][['Processing time']]                         
DesignRelated =  df[df['Coding code text'] == 'DESIGN RELATED'][['Processing time']]                        
ExternalShippingError =  df[df['Coding code text'] == 'EXTERNAL (TRANSPORTATION) SHIPPING ERROR'][['Processing time']] 
CustomerAccommodation =  df[df['Coding code text'] == 'CUSTOMER ACCOMMODATION (SALES DECISION)'][['Processing time']] 
PartCreateError =  df[df['Coding code text'] == 'PART CREATE ERROR'][['Processing time']]                    
EngineeringRelated =  df[df['Coding code text'] == 'ENGINEERING RELATED'][['Processing time']]                  
DecorationError =  df[df['Coding code text'] == 'DECORATION ERROR'][['Processing time']]                    
MasterDataError =  df[df['Coding code text'] == 'MASTER DATA ERROR'][['Processing time']]                   
ToolRoomRelated =  df[df['Coding code text'] == 'TOOL ROOM RELATED'][['Processing time']]                    
CustomerRelated =  df[df['Coding code text'] == 'CUSTOMER RELATED'][['Processing time']]                    


We can see in the below box plot that assembly related issues have more outliers may be because their count is more in number and also in assembly related issues there may be some defects that are more hard to fix which inturn consumes more time.

In [9]:
x_data = ['ASSEMBLY RELATED','MOLDING RELATED','PROCEDURE NOT FOLLOWED','UNJUSTIFIED ISSUE','ORDER ENTRY ERROR',
          'INTERNAL SHIPPING ERROR','PACKING LINE ERROR','VENDOR ERROR','INTERCO RELATED','DESIGN RELATED',
          'EXTERNAL (TRANSPORTATION) SHIPPING ERROR','CUSTOMER ACCOMMODATION (SALES DECISION)',
          'PART CREATE ERROR','ENGINEERING RELATED','DECORATION ERROR','MASTER DATA ERROR',
           'TOOL ROOM RELATED','CUSTOMER RELATED']



y_data = [AssemblyRelated['Processing time'],MoldingRelated['Processing time'],ProcedureNotFollowed['Processing time'],UnjustifiedIssue['Processing time'],
          OrderEntryError['Processing time'],InternalShippingError['Processing time'],PackingLineError['Processing time'],VendorError['Processing time'],IntercoRelated['Processing time']       
,DesignRelated['Processing time']       
,ExternalShippingError['Processing time'] 
,CustomerAccommodation['Processing time'] 
,PartCreateError['Processing time']       
,EngineeringRelated['Processing time']    
,DecorationError['Processing time']       
,MasterDataError['Processing time']       
,ToolRoomRelated['Processing time']       
,CustomerRelated['Processing time']]

fig = go.Figure()

for xd, yd in zip(x_data, y_data):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,            
            jitter=0.5,
            whiskerwidth=0.2,            
            marker_size=2,
            line_width=1)
        )

fig.update_layout(
    title='Processing Time Distribution For Each Issue',
   
    showlegend=False
)

fig.show()

Changing the date columns to datetime type and also capturing the time difference(days) between issue raised and resolved date

In [10]:
df.loc[:,'Required Start']=pd.to_datetime(df['Required Start'])
df.loc[:,'Required End']=pd.to_datetime(df['Required End'])
df.loc[:,'Changed On']=pd.to_datetime(df['Changed On'])
df.loc[:,'Completion by date']=pd.to_datetime(df['Completion by date'])
df.loc[:,'Created On']=pd.to_datetime(df['Created On'])
df.loc[:,'Malfunction start']=pd.to_datetime(df['Malfunction start'])
df.loc[:,'Notification date']=pd.to_datetime(df['Notification date'])
df.loc[:,'Purchase Order Date']=pd.to_datetime(df['Purchase Order Date'])
df.loc[:,'Reference date']=pd.to_datetime(df['Reference date'])

timeDiff = (df['Changed On']-df['Required Start'])
timeDiff.describe()

count                         895
mean     145 days 21:38:24.804469
std      215 days 04:46:42.663629
min               0 days 00:00:00
25%              45 days 00:00:00
50%              91 days 00:00:00
75%             146 days 12:00:00
max            1602 days 00:00:00
dtype: object

Next we will see how the complaint data is over the years 

In [11]:
complaintData = df[['Required Start','Complaint quantity']]
trace0 = go.Scatter(x = complaintData['Required Start'], y = complaintData['Complaint quantity'],name = 'Complaints')
data = [trace0]
layout = go.Layout(title = 'Ditribution of Customer Complaints',yaxis=dict(title='No. of Complaints'),xaxis = dict(title='Year of Complaint Made'))
fig = go.Figure(data=data,layout = layout)
iplot(fig,filename = 'complaints.html')

Next we will see how the complaints are distributed per Customer and draw a pareto chart considering the 20% of the Customers responsible for 80% of the Complaints.

In [12]:
perCustomerComplaints = df.groupby(['Customer'])[['Complaint quantity']].sum().sort_values(by='Complaint quantity',ascending=False )
perCustomerComplaints['cumSumPercent'] = (perCustomerComplaints['Complaint quantity'].cumsum()/perCustomerComplaints['Complaint quantity'].sum())*100

customerIndex = []
for index in perCustomerComplaints.index:
    customerIndex.append("Customer "+index)
perCustomerComplaints.index = customerIndex


trace0 = dict(type='bar',
              x = perCustomerComplaints.index,
              y=perCustomerComplaints['Complaint quantity'],
     marker=dict(
        color='#2196F3'
    ),
    name='Complaint quantity',
    opacity=0.8
)

trace1 = dict(type='scatter',
              x = perCustomerComplaints.index,
              y=perCustomerComplaints['cumSumPercent'],
     marker=dict(
        color='#2196F3'
    ),
    line=dict(
        color= '#263238', 
        width= 1.5),
    name=' % of Complaints per Customer', 
    xaxis = 'x1',
    yaxis='y2'
)
trace2 =  dict(type='scatter',
              x = perCustomerComplaints.index,
              y=[80]*193,
     marker=dict(
        color='#2196F3'
    ),
               
    line=dict(
        color= 'firebrick',
        dash='dash',
        width= 1.5),
    name=' 80% cutoff', 
    xaxis = 'x1',
    yaxis='y2'
)
data = [trace0,trace1,trace2]
layout = go.Layout(
    title='[Pareto Analysis] Complaint quantity vs % of  Complaints per Customer',
    plot_bgcolor='rgba(0,0,0,0)',
    legend= dict(orientation="h",
                x=0.5,
                y=1.1),
    yaxis=dict(
        
        title='No. of Complaints',
        titlefont=dict(
            color="#2196F3"
        )
    ),
    yaxis2=dict(
        title=' % of Complaints per Customer',
        titlefont=dict(
            color='#263238'
        ),
        range=[0,105],
        overlaying='y',
        anchor='x',
        side='right'
        )
    )
    


fig = go.Figure(data=data, layout=layout)
iplot(fig, filename="paretoCustomer")


Next we will see how the complaints are distributed per Issue and draw a pareto chart considering the 20% of the Issues responsible for 80% of the Complaints. We observe that Product and Unjustified issues form 80% of the Complaints. So finding reasons behind the these two Issues and what causes them can eliminate most of the complaints we might encounter in the future. 

In [13]:
complaintsPerCodeInfo = df[['Code group','Code group text','Coding code text','Complaint quantity']]
complaintsPerCodeInfo = complaintsPerCodeInfo.groupby(['Code group text'])[['Complaint quantity']].sum().sort_values(by='Complaint quantity',ascending=False)
complaintsPerCodeInfo['cumSumPercent'] = (complaintsPerCodeInfo['Complaint quantity'].cumsum()/complaintsPerCodeInfo['Complaint quantity'].sum())*100
complaintsPerCodeInfo

,Complaint quantity,cumSumPercent
Code group text,,
PRODUCT ISSUE,38627798.054,45.369
UNJUSTIFIED ISSUE,17273341.793,65.657
GMP REQUIREMENTS,13765319.113,81.825
LOGISTICS ISSUE,8829200.649,92.195
ADMINISTRATIVE ISSUE,3487983.605,96.292
INTERCO ISSUE,2312457.000,99.008
DESIGN ISSUE,794486.000,99.941
CONSUMER ISSUE,50000.000,100.000


In [14]:
trace0 = dict(type='bar',
              x = complaintsPerCodeInfo.index,
              y=complaintsPerCodeInfo['Complaint quantity'],
     marker=dict(
        color='#2196F3'
    ),
    name='Complaint quantity',
    opacity=0.8
)

trace1 = dict(type='scatter',
              x = complaintsPerCodeInfo.index,
              y=complaintsPerCodeInfo['cumSumPercent'],
     marker=dict(
        color='#2196F3'
    ),
    line=dict(
        color= '#263238', 
        width= 1.5),
    name=' % of Complaints per Issue Type', 
    xaxis = 'x1',
    yaxis='y2'
)
trace2 =  dict(type='scatter',
              x = complaintsPerCodeInfo.index,
              y=[80]*193,
     marker=dict(
        color='#2196F3'
    ),
               
    line=dict(
        color= 'firebrick',
        dash='dash',
        width= 1.5),
    name=' 80% cutoff', 
    xaxis = 'x1',
    yaxis='y2'
)
data = [trace0,trace1,trace2]
layout = go.Layout(
    title='[Pareto Analysis] Complaint quantity vs % of  Complaints per Issue Type',
    plot_bgcolor='rgba(0,0,0,0)',
    legend= dict(orientation="h",
                x=0.5,
                y=1.1),
    yaxis=dict(
        
        title='No. of Complaints',
        titlefont=dict(
            color="#2196F3"
        )
    ),
    yaxis2=dict(
        title=' % of Complaints per Issue Type',
        titlefont=dict(
            color='#263238'
        ),
        range=[0,105],
        overlaying='y',
        anchor='x',
        side='right'
        )
    )
    


fig = go.Figure(data=data, layout=layout)
iplot(fig, filename="paretoIssue")

Now we will see the Sub-Category Issues below the different Issues we discussed above and highlight the ones with max complaints.

In [15]:
df.head(2)

,Notification,Required Start,Required End,Notification Status,Priority text,Description,Material,Customer,Complaint quantity,Batch,Catalog profile,Changed On,City,Client,Code group,Code group text,Coding Code,Coding code text,Completion by date,Completion time,Country Key,Created at,Created On,Customer Material Number,Defective (external),Defective (internal),District,Malfunction start,Material Avail. Date,Notification date,Notification origin,Notification Time,Notification type,Plant for material,Postal Code,Priority,Priority type,Processing time,Processing time unit,Production Machine(s),Purchase Order Date,Purchase Order Number,Ref. quantity,Reference date,Reference number,Reference time,Region,Required End Time,Required Start Time,Return delivery qty,Sales Order,Sales order item,Sales Organization,Scrap cost,Serial Number,Start of Malfunctn (Time),Time of change,Unit of measure,Vendor Batch,Total Cost
1,200053540,2014-01-07,2014-01-28,NOCO NOTI ATCO NOPT,Major,Missing actuator within AQL - Accepted,10308487,1201843,0,NaN,QM0000001,2014-02-03,ORMOND BEACH,100,Z004,PRODUCT ISSUE,Z400,ASSEMBLY RELATED,2014-02-03,16:06:43,US,21:16:36,2014-01-07,NaN,0,0,NaN,2014-01-07,NaN,2014-01-07,Q1,21:28:42,ZC,2004,32174,2,ZQ,642.630,H,NaN,2013-01-30,2550012769,0,2014-02-03,271.150,16:06:43,FL,21:28:42,21:28:42,0,1112925,0,2000,0,NaN,21:29:47,21:06:45,PC,NaN,0.000
2,200053571,2014-01-08,2014-01-29,NOCO NOTI ATCO NOPT,Major,Incorrect box count-shortage,10311413,1009654,1170,0003285770,QM0000001,2014-05-30,LACHINE,100,NaN,NaN,NaN,NaN,2014-05-30,15:40:45,CA,14:45:07,2014-01-08,NaN,0,0,NaN,2014-01-08,2013-06-07 00:00:00,2014-01-08,Q1,14:51:57,ZC,2004,H8T 3M5,2,ZQ,3408.810,H,AS00M162,2013-04-23,02-0191752,1170,2014-05-30,187.480,13:40:45,QC,14:51:57,14:51:57,0,1127709,10,2000,0,NaN,14:57:16,13:40:49,PC,NaN,219351.600


In [16]:
codeInfo = df[['Code group','Code group text','Coding Code','Coding code text','Complaint quantity']]
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
   
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]
table = codeInfo.pivot_table(index = ['Code group','Code group text'],columns = ['Coding code text'],values = ['Complaint quantity'],aggfunc = 'sum')
table.style.apply(highlight_max,axis = 1)

Now we will consider cost factor and group complaints as per different product and issue details. Here we assumed cost to be '0' for blank cells and modified cells by eliminating string values when there is a mix of both numerical and string values. First we will consider how our Issues will get divided based on the cost they incurred.

In [50]:
TotalCostPerIssue = df.groupby(['Code group text'])[['Total Cost']].sum().sort_values('Total Cost',ascending = False)
TotalCostPerIssue = TotalCostPerIssue.groupby(['Code group text'])[['Total Cost']].sum().sort_values(by='Total Cost',ascending=False)
TotalCostPerIssue['cumSumPercent'] = (TotalCostPerIssue['Total Cost'].cumsum()/TotalCostPerIssue['Total Cost'].sum())*100
TotalCostPerIssue

,Total Cost,cumSumPercent
Code group text,,
PRODUCT ISSUE,183485747036.538,69.743
UNJUSTIFIED ISSUE,46719728336.660,87.502
LOGISTICS ISSUE,14272171190.935,92.927
ADMINISTRATIVE ISSUE,13423533575.657,98.029
DESIGN ISSUE,3063334282.500,99.193
GMP REQUIREMENTS,2109603704.490,99.995
INTERCO ISSUE,12628480.000,100.000
CONSUMER ISSUE,0.000,100.000


From the above table we can see that Product Issues cover a significant amount of cost and together with Unjustified issues have almost 88% of the cost so lets draw a pareto chart to visualize the above data.

In [53]:
trace0 = dict(type='bar',
              x = TotalCostPerIssue.index,
              y=TotalCostPerIssue['Total Cost'],
     marker=dict(
        color='#2196F3'
    ),
    name='Cost',
    opacity=0.8
)

trace1 = dict(type='scatter',
              x = TotalCostPerIssue.index,
              y=TotalCostPerIssue['cumSumPercent'],
     marker=dict(
        color='#2196F3'
    ),
    line=dict(
        color= '#263238', 
        width= 1.5),
    name=' % of Total Cost per Issue', 
    xaxis = 'x1',
    yaxis='y2'
)
trace2 =  dict(type='scatter',
              x = TotalCostPerIssue.index,
              y=[80]*193,
     marker=dict(
        color='#2196F3'
    ),
               
    line=dict(
        color= 'firebrick',
        dash='dash',
        width= 1.5),
    name=' 80% cutoff', 
    xaxis = 'x1',
    yaxis='y2'
)
data = [trace0,trace1,trace2]
layout = go.Layout(
    title='[Pareto Analysis] Cost vs  % of Total Cost per Issue',
    plot_bgcolor='rgba(0,0,0,0)',
    legend= dict(orientation="h",
                x=0.5,
                y=1.1),
    yaxis=dict(
        
        title='Cost',
        titlefont=dict(
            color="#2196F3"
        )
    ),
    yaxis2=dict(
        title='  % of Total Cost per Issue',
        titlefont=dict(
            color='#263238'
        ),
        range=[0,105],
        overlaying='y',
        anchor='x',
        side='right'
        )
    )
    


fig = go.Figure(data=data, layout=layout)
iplot(fig, filename="paretoCustomer")
